# 02 Features basics

Let's start with an existing analysis configuration, that we copy into a temporary location to be used as working directory.

In [ ]:
from pathlib import Path
import tempfile
from blueetl.utils import copy_config

workdir = tempfile.TemporaryDirectory(suffix="_blueetl")
workdir_path = Path(workdir.name)

config_file = workdir_path / "config.yaml"
copy_config("../data/analysis/config.yaml", config_file)

# print(config_file)
# print(config_file.read_text())

We can now initialize a MultiAnalyzer object with the following code, where you can specify different parameters if needed:

In [ ]:
from blueetl.analysis import run_from_file

ma = run_from_file(
    config_file,
    extract=False,
    calculate=False,
    show=False,
    clear_cache=True,
    loglevel="ERROR",
)
print(ma)

Since we passed `extract=False` to the previous call, we have to extract the repository explicitly:

In [ ]:
ma.extract_repo()

And since we passed `calculate=False` to the previous call, we have to calculate the features explicitly:

In [ ]:
ma.calculate_features()

We can now inspect the list of analyses in the MultiAnalyzer object:

In [ ]:
ma.names

and access each of them as an Analyzer object:

In [ ]:
ma.spikes

Each Analyzer object provides two special attributes: `repo` and `features`, that can be used to access the extracted data and the calculated features.

You can inspect the list of extracted and calculated DataFrames calling `names` on them, as shown below:

In [ ]:
ma.spikes.repo.names

In [ ]:
ma.spikes.features.names

You can access the wrapped DataFrames using the `df` attribute on each object:

In [ ]:
ma.spikes.repo.report.df

The DataFrames of features can be accessed in the same way:

In [ ]:
ma.spikes.features.by_neuron_class_0_0__0.df

and in this case also the `attrs` dictionary attached to the DataFrame is populated with the parameters used for the computation:

In [ ]:
ma.spikes.features.by_neuron_class_0_0__0.df.attrs

The parameters have been automatically calculated combining `params`, `params_product`, and `params_zip` from the original configuration.

In this case, it may be convenient to access a single DataFrame contaning the concatenation of the features of the same type, where the varying parameters are added as new columns.

The name of the DataFrame is the same as the split DataFrames, without the suffix:

In [ ]:
ma.spikes.features.by_neuron_class.df

Note that the column names in the previous DataFrame have been shortened. You can see the full names in the `aliases` DataFrame:

In [ ]:
ma.spikes.features.by_neuron_class.aliases

You can also inspect all the parameters that were used for the computation, accessing the `params` attribute:

In [ ]:
ma.spikes.features.by_neuron_class.params

During the extraction and computation, some files have been created to be used as cache.

Usually you don't need to access them directly, and if they are deleted they will be created again at the next run.

They may be automatically deleted when the cache is invalidated.

In [ ]:
!cd {workdir.name} && tree

You can remove the full working directory if you don't need it anymore:

In [ ]:
workdir.cleanup()